OpenAI´s function_Calling was first deprecated in favor of tool_calling - but now the correct terms seems to be function_calling again.
If you call it tool- or function calling - at the moment is does not really matter. 

Reference: https://platform.openai.com/docs/guides/function-calling

In [16]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [17]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [19]:
print(llm.invoke("How will the weather be in munich today?"))

content="I'm unable to provide real-time weather updates. I recommend checking a reliable weather website or app for the most current weather information in Munich." additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 17, 'total_tokens': 44, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BxAPkbZuuiwXUtB7QqHX5F349ZWgs', 'finish_reason': 'stop', 'logprobs': None} id='run--87e6a5eb-4246-435d-900f-ac9067b784db-0' usage_metadata={'input_tokens': 17, 'output_tokens': 27, 'total_tokens': 44, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [20]:
from langchain_core.tools import tool


@tool
def fake_weather_api(city: str) -> str:
    """
    Check the weather in a specified city.

    Args:
        city (str): The name of the city where you want to check the weather.

    Returns:
        str: A description of the current weather in the specified city.
    """
    return "Sunny, 22°C"


@tool
def outdoor_seating_availability(city: str) -> str:
    """
    Check if outdoor seating is available at a specified restaurant in a given city.

    Args:
        city (str): The name of the city where you want to check for outdoor seating availability.

    Returns:
        str: A message stating whether outdoor seating is available or not.
    """
    return "Outdoor seating is available."


tools = [fake_weather_api, outdoor_seating_availability]

In [22]:
outdoor_seating_availability.description

'Check if outdoor seating is available at a specified restaurant in a given city.\n\nArgs:\n    city (str): The name of the city where you want to check for outdoor seating availability.\n\nReturns:\n    str: A message stating whether outdoor seating is available or not.'

In [5]:
# from langchain_core.pydantic_v1 import BaseModel, Field

# class WeatherCheck(BaseModel):
#     """Check the weather in a specified city."""

#     city: str = Field(..., description="Name of the city to check the weather for")


# class OutdoorSeatingCheck(BaseModel):
#     """Check if outdoor seating is available at a specified restaurant in a given city."""

#     city: str = Field(..., description="Name of the city where the restaurant is located")


# tools = [WeatherCheck, OutdoorSeatingCheck]

In [21]:
llm_with_tools = llm.bind_tools(tools)

In [23]:
result = llm_with_tools.invoke("How will the weather be in munich today?")
print(result)

content='' additional_kwargs={'tool_calls': [{'id': 'call_7QFbfuLCRqc5WRb7YgGMLTga', 'function': {'arguments': '{"city":"Munich"}', 'name': 'fake_weather_api'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 158, 'total_tokens': 174, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-BxBSShOEHPgEqf3CnI79kbPmfa8XY', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--272b014d-9b1c-4c9a-834b-bc0efb61fdad-0' tool_calls=[{'name': 'fake_weather_api', 'args': {'city': 'Munich'}, 'id': 'call_7QFbfuLCRqc5WRb7YgGMLTga', 'type': 'tool_call'}] usage_metadata={'input_tokens': 158, 'output_tokens': 16, 'total_tokens': 174, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'ou

In [24]:
result = llm_with_tools.invoke(
    "How will the weather be in munich today? Do you still have seats outdoor available?"
)
print(result)

content='' additional_kwargs={'tool_calls': [{'id': 'call_P6pVWLxQXSCHeXQZSte3D9iV', 'function': {'arguments': '{"city": "munich"}', 'name': 'fake_weather_api'}, 'type': 'function'}, {'id': 'call_qYIJFK3aD5aCTgg9LOAXCoWQ', 'function': {'arguments': '{"city": "munich"}', 'name': 'outdoor_seating_availability'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 166, 'total_tokens': 217, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-BxBTWqAAWjhBYeysOQ7K4q3GLgdHm', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--923c789b-68a5-4758-bdb2-7ea133399fdb-0' tool_calls=[{'name': 'fake_weather_api', 'args': {'city': 'munich'}, 'id': 'call_P6pVWLxQXSCHeXQZSte3D9iV', 'type':

In [25]:
result.tool_calls

[{'name': 'fake_weather_api',
  'args': {'city': 'munich'},
  'id': 'call_P6pVWLxQXSCHeXQZSte3D9iV',
  'type': 'tool_call'},
 {'name': 'outdoor_seating_availability',
  'args': {'city': 'munich'},
  'id': 'call_qYIJFK3aD5aCTgg9LOAXCoWQ',
  'type': 'tool_call'}]

In [26]:
from langchain_core.messages import HumanMessage, ToolMessage

messages = [
    HumanMessage(
        "How will the weather be in munich today? I would like to eat outside if possible"
    )
]
llm_output = llm_with_tools.invoke(messages)
messages.append(llm_output)

In [27]:
messages

[HumanMessage(content='How will the weather be in munich today? I would like to eat outside if possible', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_pVUT5yFMlOIz28Ci83zLpKnc', 'function': {'arguments': '{"city": "Munich"}', 'name': 'fake_weather_api'}, 'type': 'function'}, {'id': 'call_Kj1ddugTdlEx4qrZTSSDPWD2', 'function': {'arguments': '{"city": "Munich"}', 'name': 'outdoor_seating_availability'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 166, 'total_tokens': 217, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-BxBTpQTCLbKo6MxpqotENyRon0iUI', 'finish_reason': 'tool_calls', 'logprobs': None},

In [28]:
tool_mapping = {
    "fake_weather_api": fake_weather_api,
    "outdoor_seating_availability": outdoor_seating_availability,
}

In [29]:
for tool_call in llm_output.tool_calls:
    tool = tool_mapping[tool_call["name"].lower()]
    tool_output = tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

In [30]:
messages

[HumanMessage(content='How will the weather be in munich today? I would like to eat outside if possible', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_pVUT5yFMlOIz28Ci83zLpKnc', 'function': {'arguments': '{"city": "Munich"}', 'name': 'fake_weather_api'}, 'type': 'function'}, {'id': 'call_Kj1ddugTdlEx4qrZTSSDPWD2', 'function': {'arguments': '{"city": "Munich"}', 'name': 'outdoor_seating_availability'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 166, 'total_tokens': 217, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-BxBTpQTCLbKo6MxpqotENyRon0iUI', 'finish_reason': 'tool_calls', 'logprobs': None},

In [31]:
llm_with_tools.invoke(messages)

AIMessage(content='The weather in Munich today is sunny with a temperature of 22°C. Additionally, outdoor seating is available, so you can enjoy your meal outside!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 245, 'total_tokens': 276, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-BxBUGq4BVfO7iI9sSIDbgxOFKQL79', 'finish_reason': 'stop', 'logprobs': None}, id='run--36d269ab-bf79-485b-b406-000a7bdd957b-0', usage_metadata={'input_tokens': 245, 'output_tokens': 31, 'total_tokens': 276, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})